In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error, explained_variance_score
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge
from sklearn.model_selection import train_test_split

from regressors import stats
import numpy as np

In [2]:
data = pd.read_csv('cmm_erdos_2022/pharmacy_tx.csv')

In [3]:
data = data[data.rejected == False] # we are not taking rejected claims

# splitting data

train,test= train_test_split(data,test_size=0.2)

In [8]:
train_temp=train
test_temp=test

We are going to do feature engineering and introduce new features in our data. Here we introduce 7 features `p1`,...,`p7` by aggregating over various columns.

In [9]:
temp = train_temp.groupby(['drug','pharmacy','bin'],as_index=False).patient_pay.mean()
temp['p1'] = temp['patient_pay']
del temp['patient_pay']

train_temp = pd.merge(train_temp,temp,on=['drug','pharmacy','bin'], how='left')

test_temp = pd.merge(test_temp,temp,on=['drug','pharmacy','bin'], how='left')


temp = train_temp.groupby(['drug','pharmacy','group'],as_index=False).patient_pay.mean()
temp['p2'] = temp['patient_pay']
del temp['patient_pay']

train_temp = pd.merge(train_temp,temp,on=['drug','pharmacy','group'], how='left')
train_temp['p2']=train_temp['p2'].fillna(train_temp['p1'])

test_temp = pd.merge(test_temp,temp,on=['drug','pharmacy','group'], how='left')

temp = train_temp.groupby(['drug','pharmacy','diagnosis'],as_index=False).patient_pay.mean()
temp['p3'] = temp['patient_pay']
del temp['patient_pay']

train_temp = pd.merge(train_temp,temp,on=['drug','pharmacy','diagnosis'], how='left')

test_temp = pd.merge(test_temp,temp,on=['drug','pharmacy','diagnosis'], how='left')



temp = train_temp.groupby(['drug','pharmacy'],as_index=False).patient_pay.mean()
temp['p4'] = temp['patient_pay']
del temp['patient_pay']

train_temp = pd.merge(train_temp,temp,on=['drug','pharmacy'], how='left')

test_temp = pd.merge(test_temp,temp,on=['drug','pharmacy'], how='left')

temp = train_temp.groupby(['drug','pharmacy','pcn'],as_index=False).patient_pay.mean()
temp['p5'] = temp['patient_pay']
del temp['patient_pay']

train_temp = pd.merge(train_temp,temp,on=['drug','pharmacy','pcn'], how='left')
train_temp['p5']=train_temp['p5'].fillna(train_temp['p1'])

test_temp = pd.merge(test_temp,temp,on=['drug','pharmacy','pcn'], how='left')

temp = train_temp.groupby(['drug','diagnosis'],as_index=False).patient_pay.mean()
temp['p6'] = temp['patient_pay']
del temp['patient_pay']

train_temp = pd.merge(train_temp,temp,on=['drug','diagnosis'], how='left')
test_temp = pd.merge(test_temp,temp,on=['drug','diagnosis'], how='left')

temp = train_temp.groupby(['drug','pcn'],as_index=False).patient_pay.mean()
temp['p7'] = temp['patient_pay']
del temp['patient_pay']

train_temp = pd.merge(train_temp,temp,on=['drug','pcn'], how='left')

test_temp = pd.merge(test_temp,temp,on=['drug','pcn'], how='left')


In [18]:
train_temp['p7']=train_temp['p7'].fillna(train_temp['p1'])

train_temp2=train_temp[['p1','p2','p3','p4','p5','p6','p7','patient_pay']]
train_temp2=train_temp2.drop_duplicates()

X_train=train_temp2[['p1','p2','p3','p4','p5','p6','p7']]
y_train=train_temp2['patient_pay']

# checking how many empty entries we have in test set
print(test_temp.isna().sum())

# imputing the test data with other non-null entries
test_temp['p1']=test_temp['p1'].fillna(test_temp['p6'])
test_temp['p2']=test_temp['p2'].fillna(test_temp['p1'])
test_temp['p3']=test_temp['p3'].fillna(test_temp['p1'])
test_temp['p5']=test_temp['p5'].fillna(test_temp['p1'])
test_temp['p7']=test_temp['p7'].fillna(test_temp['p1'])


X_test=test_temp[['p1','p2','p3','p4','p5','p6','p7']]
y_test=test_temp['patient_pay']

tx_date             0
pharmacy            0
diagnosis           0
drug                0
bin                 0
pcn            664797
group          724059
rejected            0
patient_pay         0
p1                  0
p2                  0
p3                  0
p4                  0
p5                  0
p6                  0
p7                  0
pred                0
dtype: int64


In [32]:
#Ridge Regression on training set

lreg = Ridge(alpha=10)
lreg.fit(np.log(X_train),np.log(y_train))

train_predict = np.exp(lreg.predict(np.log(X_train)))

print(r2_score(train_predict,y_train))
print(mean_absolute_error(train_predict,y_train))
print(mean_squared_error(train_predict,y_train))

#stats.summary(lreg,np.log(X_train),np.log(y_train))
lreg.coef_

0.9719422242576868
4.506996089800478
95.88962106903575


array([-0.2648703 ,  0.74314957,  0.09953541, -0.12611109,  0.41130399,
        0.0694566 ,  0.08223285])

In [33]:
test_temp2= test_temp.drop_duplicates()

X_test=test_temp2[['p1','p2','p3','p4','p5','p6','p7']]
y_test=test_temp2['patient_pay']

#predictions
pred = np.exp(lreg.predict(np.log(X_test)))

print(r2_score(pred,y_test))
print(mean_absolute_error(pred,y_test))
print(mean_squared_error(pred,y_test))

0.9817531930875331
2.298403563454222
32.76153956994347


In [24]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

rf.fit(X_train,y_train)

RandomForestRegressor()

In [29]:
#predictions
pred = rf.predict(X_test)

print(r2_score(pred,y_test))
print(mean_absolute_error(pred,y_test))
print(mean_squared_error(pred,y_test))

0.9887744786590821
1.4587564457906517
19.76374275702966


In [31]:
test_temp2['pred']=pred

# entries where both group and pcn are missing

test_temp2[(abs(test_temp2.patient_pay - test_temp2.pred)>50)]

/var/folders/58/fw1mf93d4gxg7d40q7fc5hj00000gn/T/ipykernel_6048/1021757078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_temp2['pred']=pred


,tx_date,pharmacy,diagnosis,drug,bin,pcn,group,rejected,patient_pay,p1,p2,p3,p4,p5,p6,p7,pred
40490,2022-01-25,Pharmacy #3,Q85.91,branded plazamiglutic,664344,BIZF,QK6BI1N61,False,120.17,49.919138,63.520000,66.719925,63.997389,63.520000,63.518349,66.748260,64.439833
41426,2022-01-04,Pharmacy #53,U06.52,branded tugesutin,725700,NaN,DYGBI610ZY,False,254.20,214.446145,191.830000,156.612667,166.730469,214.446145,158.577698,214.446145,191.838900
53279,2022-03-16,Pharmacy #16,Q72.66,branded tugesutin,725700,NaN,DYGBI610ZY,False,254.20,203.305251,192.440000,160.548796,162.961182,203.305251,161.541638,203.305251,192.403300
56549,2022-12-05,Pharmacy #8,Z20.23,branded flacelfatastat,664344,IF448,FZPLF4O6FD,False,161.96,224.712203,224.712203,227.001343,228.810103,224.712203,205.181796,165.346358,237.385147
59840,2022-01-10,Pharmacy #51,I91.91,branded prefluflomycin,725700,327CKV,IOEAN1DWVV3Y,False,359.85,295.757727,299.817568,270.542727,240.089927,299.817568,229.393074,299.454793,306.089350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437987,2022-11-13,Pharmacy #15,C14.36,branded prefluflomycin,322463,T52GV,EVD4X5,False,16.29,137.669750,137.669750,186.890971,220.794780,137.669750,232.684311,16.290000,120.289681
2468514,2022-01-18,Pharmacy #19,U06.52,branded tugesutin,96934,S76J7V6,NaN,False,232.08,151.543193,151.543193,165.929912,167.933114,178.170101,158.577698,179.030645,179.798600
2480615,2022-01-15,Pharmacy #52,Z20.23,branded vivafastat,96934,S76J7V6,NaN,False,227.62,147.643448,147.643448,151.479686,153.612466,174.290000,150.504578,175.757456,174.557967
2512713,2022-02-17,Pharmacy #9,Z66.42,branded nusudaric,571569,W7L3,V96T9QL5,False,110.62,160.094286,58.860000,119.327231,118.140395,58.860000,119.567641,62.793881,58.871100
